In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from collections import deque
import random
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Dropout,BatchNormalization
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
import tensorflow as tf

SEQ_LEN=60
FUTURE_PERIOD_PREDICT=3
RATIO_TO_PREDICT="LTC-USD"
EPOCHS=10
BATCHSIZE=64
NAME=f"{RATIO_TO_PREDICT}--{SEQ_LEN}--seq--{FUTURE_PERIOD_PREDICT}--PRED__{int(time.time())}"


In [16]:
df=pd.read_csv("C:/Users/91789/ML Projects/Crypto Price Analysis/crypto_data/LTC-USD.csv", names=["time","low","high","open","close","volume"])


def classify(current,future):
    if float(future)>float(current):
        return 1
    else:
        return 0
print(df.values)

[[1.52896866e+09 9.65800020e+01 9.65899960e+01 9.65899960e+01
  9.65800020e+01 9.64720000e+00]
 [1.52896872e+09 9.64499970e+01 9.66699980e+01 9.65899960e+01
  9.66600040e+01 3.14387024e+02]
 [1.52896878e+09 9.64700010e+01 9.65700000e+01 9.65700000e+01
  9.65700000e+01 7.71297990e+01]
 ...
 [1.53521514e+09 5.80200000e+01 5.80200000e+01 5.80200000e+01
  5.80200000e+01 6.95349700e+00]
 [1.53521520e+09 5.80200000e+01 5.80800020e+01 5.80200000e+01
  5.80800020e+01 2.02403183e+02]
 [1.53521526e+09 5.80700000e+01 5.80999980e+01 5.80700000e+01
  5.80900000e+01 1.60602554e+02]]


In [17]:
def preprocess_df(df):
    df=df.drop("future",1)
    for col in df.columns:
        if col!="target":
            df[col]=df[col].pct_change()
            df.dropna(inplace=True)
            df[col]=preprocessing.scale(df[col].values)
    df.dropna(inplace=True)
    
    sequential_data=[]
    
    prev_days=deque(maxlen=SEQ_LEN)  
    
    for i in df.values:    
        prev_days.append([n for n in i[:-1]])  
        if len(prev_days)==SEQ_LEN:
            sequential_data.append([np.array(prev_days),i[-1]])
    random.shuffle(sequential_data)
    buys=[]
    sells=[]
    
    for seq,target in sequential_data:
        if target==0:
            sells.append([seq,target])
        elif target==1:
            buys.append([seq,target])
            
    random.shuffle(sells)
    random.shuffle(buys)
    
    lower=min(len(buys),len(sells))
    
    buys=buys[:lower]
    sells=sells[:lower]
    
    sequential_data=buys+sells
    random.shuffle(sequential_data)
    
    X=[]
    y=[]
    
    for seq,target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X),y
    
    
            
    
    
    

#df.head()

In [18]:
main_df=pd.DataFrame()

ratios=['BCH-USD','BTC-USD','ETH-USD','LTC-USD']
for ratio in ratios:
    dataset=f"C:/Users/91789/OneDrive/Documents/crypto_data/crypto_data/{ratio}.csv"
    
    df=pd.read_csv(dataset,names=["time","low","high","open","close","volume"])
    
    df.rename(columns={"close":f'{ratio}_close',"volume":f"{ratio}_volume"},inplace=True)
    df.set_index("time",inplace=True)
    df=df[[f'{ratio}_close',f"{ratio}_volume"]]
    
    if len(main_df)==0:
        main_df=df
    else:
        main_df=main_df.join(df)
        
main_df.fillna(method="ffill", inplace=True)  
main_df.dropna(inplace=True)
main_df.head()

,BCH-USD_close,BCH-USD_volume,BTC-USD_close,BTC-USD_volume,ETH-USD_close,ETH-USD_volume,LTC-USD_close,LTC-USD_volume
time,,,,,,,,
1528968720,870.859985,26.856577,6487.379883,7.706374,486.01001,26.019083,96.660004,314.387024
1528968780,870.099976,1.124300,6479.410156,3.088252,486.00000,8.449400,96.570000,77.129799
1528968840,870.789978,1.749862,6479.410156,1.404100,485.75000,26.994646,96.500000,7.216067
1528968900,870.000000,1.680500,6479.979980,0.753000,486.00000,77.355759,96.389999,524.539978
1528968960,869.989990,1.669014,6480.000000,1.490900,486.00000,7.503300,96.519997,16.991997


In [19]:
main_df["future"]=main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

In [20]:
main_df[[f"{RATIO_TO_PREDICT}_close",'future']].head()   

,LTC-USD_close,future
time,,
1528968720,96.660004,96.389999
1528968780,96.570000,96.519997
1528968840,96.500000,96.440002
1528968900,96.389999,96.470001
1528968960,96.519997,96.400002


In [21]:
main_df["target"]=list(map(classify,main_df[f"{RATIO_TO_PREDICT}_close"],main_df["future"]))
main_df[[f"{RATIO_TO_PREDICT}_close",'future',"target"]].head(10) 

,LTC-USD_close,future,target
time,,,
1528968720,96.660004,96.389999,0
1528968780,96.570000,96.519997,0
1528968840,96.500000,96.440002,0
1528968900,96.389999,96.470001,1
1528968960,96.519997,96.400002,0
1528969020,96.440002,96.400002,0
1528969080,96.470001,96.400002,0
1528969140,96.400002,96.400002,0
1528969200,96.400002,96.400002,0


In [22]:
times=sorted(main_df.index.values)
last_5pct=times[-int(0.05*len(times))]
print(last_5pct)

1534879920


In [23]:
validation_main_df=main_df[(main_df.index>=last_5pct)]
main_df=main_df[(main_df.index<last_5pct)]

In [24]:
train_x,train_y=preprocess_df(main_df)
test_x,test_y=preprocess_df(validation_main_df)


C:\Users\91789\AppData\Local\Temp\ipykernel_18268\468924936.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop("future",1)
C:\Users\91789\AppData\Local\Temp\ipykernel_18268\468924936.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop("future",1)


In [26]:
train_x,train_y=preprocess_df(main_df)
test_x,test_y=preprocess_df(validation_main_df)

C:\Users\91789\AppData\Local\Temp\ipykernel_18268\468924936.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop("future",1)
C:\Users\91789\AppData\Local\Temp\ipykernel_18268\468924936.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df=df.drop("future",1)


In [27]:
train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
test_x = np.asarray(test_x)
test_y = np.asarray(test_y)

In [28]:
print(f"train data:{len(train_x)},test_x:{len(test_x)}")
#print(f"Don't buys:{train_y.count(0)},buys:{train_y.count(1)}")
#print(f"Validation Don't Buys:{test_y.count(0)},VAlidation buys:{test_y.count(1)}")

train data:75008,test_x:3810


In [29]:
model=Sequential()

model.add(LSTM(128,input_shape=(train_x.shape[1:]),return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128,return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2,activation='softmax'))

opt=tf.keras.optimizers.Adam(lr=0.001)

model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

tensorboard=TensorBoard(log_dir=f"logs/{NAME}")

filepath="RNN_Final--{epoch:02d}-{val_accuracy:.3f}"
checkpoint=ModelCheckpoint('models/{}.model'.format(filepath,monitor='val_accuracy',verbose=1,mode='max',save_best_only=True))

history=model.fit(train_x,train_y,batch_size=BATCHSIZE,epochs=EPOCHS,validation_data=(test_x,test_y),
                  callbacks=[tensorboard,checkpoint])

score=model.evaluate(test_x,test_y,verbose=0)

print('Test_loss:',score[0])
print('Test_accuracy:',score[1])

model.save("models/{}".format(NAME))



Epoch 1/10
1172/1172 [==============================] - ETA: 0s - loss: 0.7079 - accuracy: 0.5099

INFO:tensorflow:Assets written to: models\RNN_Final--01-0.526.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final--01-0.526.model\assets


1172/1172 [==============================] - 419s 351ms/step - loss: 0.7079 - accuracy: 0.5099 - val_loss: 0.6896 - val_accuracy: 0.5260
Epoch 2/10
1172/1172 [==============================] - ETA: 0s - loss: 0.6904 - accuracy: 0.5308

INFO:tensorflow:Assets written to: models\RNN_Final--02-0.542.model\assets


...

INFO:tensorflow:Assets written to: models\RNN_Final--10-0.576.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final--10-0.576.model\assets


1172/1172 [==============================] - 1724s 1s/step - loss: 0.6845 - accuracy: 0.5556 - val_loss: 0.6774 - val_accuracy: 0.5764
Test_loss: 0.6773590445518494
Test_accuracy: 0.5763779282569885


INFO:tensorflow:Assets written to: models/LTC-USD--60--seq--3--PRED__1692256161\assets


INFO:tensorflow:Assets written to: models/LTC-USD--60--seq--3--PRED__1692256161\assets
